# Practicum 9: Demand - Solar rooftop

In this lecture, we talked about net metering policies and how they impact the profitability of solar panels along income groups.
 
We will consider the general equilibrium effects of solar rooftop using our model.

*Note 1*: This will be a limited simulation with only one representative consumer. To make this more interesting, we should have different types of consumers and income groups. 

*Note 2*: Allowing for different types of consumers also shows good and bad selection: under good NEM policies, the "good" type of consumers who consume when solar is coincident are encouraged to enter.

We load the libraries and set the path.

In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyomo.environ import *

In [173]:
dirpath = "/Users/marreguant/Dropbox/TEACHING/BSE/Electricity2026/day9/practicum/"

## Building the model

We load the same data as usual, and also clean it up to simplify it further and create the demand and import curves.

In [174]:
TECHS = ["hydronuc","gas1","gas2","gas3","newgas","wind","solar"]
INV_TECHS = ["newgas","wind","solar"]
RES_TECHS = ["wind","solar"]

In [175]:
def load_and_prepare(dirpath: str):

    dfclust = pd.read_csv(f"{dirpath}/data_jaere_clustered.csv")
    tech = pd.read_csv(f"{dirpath}/data_technology.csv")

    # Re-scaling weights: multiply by 8.76 and normalize (so that we can interpret it as annual revenue in million USD)
    dfclust["weights"] = 8.76 * dfclust["weights"] / dfclust["weights"].sum()

    # Here only one demand type to make it easier
    dfclust["demand"] = dfclust["q_residential"] + dfclust["q_commercial"] + dfclust["q_industrial"]

    # Calibrate imports: imports = am + bm * price
    elas = np.array([0.1, 0.2, 0.5, 0.3])
    dfclust["bm"] = elas[3] * dfclust["imports"] / dfclust["price"]
    dfclust["am"] = dfclust["imports"] - dfclust["bm"] * dfclust["price"]

    # Set index names for tech
    tech.index = TECHS
    
        # Annualization factor and fixed costs
    afactor = (1 - (1 / (1.05**20.0))) / 0.05
    tech["F"]  = tech["F"]  / afactor
    tech["F2"]  = tech["F2"]  / (2*afactor)

    return dfclust, tech

dfclust, tech = load_and_prepare(dirpath)

In [176]:
tech

,techname,heatrate,heatrate2,F,capLB,capUB,new,renewable,solar,thermal,e,e2,c,c2,F2
hydronuc,Hydro/Nuclear,10.000000,0.000000,0.000000,1.000,1.000,0,0,0,0,0.000000,0.000000,10.000000,0.000000,0.000000
gas1,Existing 1,6.671990,0.092912,0.000000,7.500,7.500,0,0,0,1,0.360184,0.004886,23.351965,0.325193,0.000000
gas2,Existing 2,9.794118,0.286247,0.000000,10.500,10.500,0,0,0,1,0.546134,0.011078,34.279413,1.001866,0.000000
gas3,Existing 3,13.818120,20.535160,0.000000,0.578,0.578,0,0,0,1,0.816768,0.234476,48.363420,71.873060,0.000000
newgas,New Gas,6.600000,0.000000,78.477250,0.000,100.000,1,0,0,1,0.350000,0.000000,23.351965,0.325193,0.784773
wind,Wind,0.000000,0.000000,100.303234,0.000,100.000,1,1,0,0,0.000000,0.000000,0.000000,0.000000,1.003032
solar,Solar,0.000000,0.000000,100.303234,0.000,100.000,1,1,1,0,0.000000,0.000000,0.000000,0.000000,1.003032


For this exercise, we use demand and a cost of $40/MWh of grid costs to estimate the annual cost of grid services. This is relevant to the issue of net-metering. If everyone goes off-grid, who pays for transmission?

In [177]:
grid_costs = sum(dfclust["weights"] * dfclust["q_residential"]) * 40.0
print(f"Grid costs: {grid_costs:.2f} million USD/year")

Grid costs: 3713.82 million USD/year


## Adding net-metering

We will use a simplified version of the model with only solar panels.

All consumers will pay a flat tariff to make the issue of net metering more pervasive. There is also a certain amount of grid costs that need to be recovered.

Consumers will be able to net their demand out fully if `nem` equals 1. However, they will only receive the wholesale price if `nem` equals 0 for their non-coincident demand.

In [178]:
def clear_market_rooftop(data, tech, ng_price=3.5, 
            pretail=50.0, kw = 3.0, hh=12.0, nem=0,
            Kmax=50.0, solver_name="ipopt"):
    """
    Welfare max with endogenous investment (NLP)
    """
    T = len(data)
    S = 3 # demand segments

    # plain python lookups (no Params)
    c = {k: float(tech.at[k, "c"]) for k in TECHS}
    if "heatrate" in tech.columns:
        for k in ["gas1","gas2","gas3","newgas"]:
            if k in tech.index:
                c[k] = float(tech.at[k, "heatrate"]) * float(ng_price)

    e = {k: float(tech.at[k, "e"]) for k in TECHS} if "e" in tech.columns else {k: 0.0 for k in TECHS}
    F = {k: float(tech.at[k, "F"]) for k in TECHS} if "F" in tech.columns else {k: 0.0 for k in TECHS}
    F2 = {k: float(tech.at[k, "F2"]) for k in TECHS} if "F2" in tech.columns else {k: 0.0 for k in TECHS}
    capUB = {k: float(tech.at[k, "capUB"]) for k in ["gas1","gas2","gas3"] if "capUB" in tech.columns and k in tech.index}

    # assume T = number of rows (time), S = number of demand segments
    elas = [.1, .2, .5, .3]  # residential, commercial, industrial, imports
    a = np.zeros((T, S))
    b = np.zeros((T, S))

    p = np.asarray(data["price"])  # or data.price if it's an object with attributes

    # Helper to keep things readable
    def calibrate_linear_demand(q, elasticity, price):
        """
        Calibrate q = a - b*price with constant elasticity at the observed point:
        elasticity = (dq/dp) * (p/q)  =>  dq/dp = elasticity * (q/p)
        We store slope 'b' (positive) so demand is q = a - b*p.
        """
        q = np.asarray(q)
        slope = elasticity * q / price          # pointwise dq/dp (likely negative)
        slope = np.mean(slope) * np.ones_like(q) # make slope constant over t
        intercept = q + slope * price            # because q = a - b*p and slope is dq/dp
        return intercept, slope

    # Residential 
    a[:, 0], b[:, 0] = calibrate_linear_demand(
        q=data["q_residential"], elasticity=elas[0], price=p
    )

    # Commercial 
    a[:, 1], b[:, 1] = calibrate_linear_demand(
        q=data["q_commercial"], elasticity=elas[1], price=p
    )

    # Industrial 
    a[:, 2], b[:, 2] = calibrate_linear_demand(
        q=data["q_industrial"], elasticity=elas[2], price=p
    )

    # Defining self consumption
    selfc = [min([data.solar_cap[t]*kw, data.q_residential[t]/hh]) for t in range(T)]
    gout  = [max([data.solar_cap[t]*kw - data.q_residential[t]/hh, 0.0]) for t in range(T)]

    m = ConcreteModel()
    m.T = RangeSet(0, T-1)
    m.S = RangeSet(0, S-1)
    m.I = Set(initialize=TECHS, ordered=False)
    m.J = Set(initialize=INV_TECHS, ordered=False)

    m.price   = Var(m.T, domain=Reals)
    m.demand  = Var(m.T, m.S, domain=Reals)
    m.imports = Var(m.T, domain=Reals)

    m.q     = Var(m.T, m.I, domain=NonNegativeReals)
    m.costs = Var(m.T, domain=Reals)
    m.gs    = Var(m.T, domain=Reals)

    m.K      = Var(m.J, bounds=(0.0, float(Kmax)))  # capacity investments

    # Constraint wind and newgas to zero to focus on solar only
    m.zero_newgas = Constraint(expr = m.K["newgas"] == 0.0)
    m.zero_wind = Constraint(expr = m.K["wind"] == 0.0)

    # Max investment in solar as number of households * kw per household
    m.cap_solar_inv = Constraint(expr = m.K["solar"] <= hh * kw)

    # Objective: gross surplus - variable costs - fixed costs
    if nem == 0:
        m.obj = Objective(
            expr=sum(data.weights[t] * (m.gs[t] - m.costs[t]) for t in m.T)
                 - sum(F.get(k) * m.K[k]  + F2.get(k) * m.K[k]**2 for k in m.J),
            sense=maximize
        )
    elif nem == 1:
        m.obj = Objective(
            expr=sum(data.weights[t] * (m.gs[t] - m.costs[t]) for t in m.T)
                 - sum(F.get(k) * m.K[k] + F2.get(k) * m.K[k]**2 for k in m.J) 
                 + sum(data.weights[t] * (pretail - m.price[t]) * data.solar_cap[t] * m.K["solar"] for t in m.T),
            sense=maximize
        )
    elif nem == 2:
        m.obj = Objective(
            expr=sum(data.weights[t] * (m.gs[t] - m.costs[t]) for t in m.T)
                 - sum(F.get(k) * m.K[k] + F2.get(k) * m.K[k]**2 for k in m.J) 
                 + sum(data.weights[t] * (pretail - m.price[t]) * selfc[t] * m.K["solar"]/kw for t in m.T),
            sense=maximize
        )
    elif nem == 3:
        m.obj = Objective(
            expr=sum(data.weights[t] * (m.gs[t] - m.costs[t]) for t in m.T)
                 - sum(F.get(k) * m.K[k] + F2.get(k) * m.K[k]**2 for k in m.J)
                 + sum(data.weights[t] * (pretail - m.price[t]) * selfc[t] * m.K["solar"]/kw for t in m.T)
                 - sum(data.weights[t] * m.price[t] * gout[t] * m.K["solar"]/ kw for t in m.T),
            sense=maximize
        )
    else:
        raise ValueError("nem must be 0, 1, 2, or 3")
    
    # Market clearing
    m.demand_curve_0  = Constraint(m.T, rule=lambda m,t: m.demand[t,0]  == data.q_residential[t])
    m.demand_curve_1  = Constraint(m.T,rule=lambda m,t: m.demand[t,1]  == a[t,1] - b[t,1] * m.price[t])
    m.demand_curve_2 = Constraint(m.T,rule=lambda m,t: m.demand[t,2]  == a[t,2] - b[t,2] * m.price[t])
    m.imports_curve = Constraint(m.T, expr={t: m.imports[t] == data.am[t] + data.bm[t] * m.price[t] for t in m.T})
    m.market_clear  = Constraint(m.T, expr={t: sum(m.demand[t, s] for s in m.S)  == sum(m.q[t,k] for k in m.I) + m.imports[t] for t in m.T})

    # Surplus + costs
    m.surplus_def = Constraint(
        # Note: now surprlus only matters for sensitive households (rest fixed)
        m.T, rule = lambda m,t:  m.gs[t] == 
        sum(m.price[t] * (a[t,s] -  b[t,s] * m.price[t])
                      + ((a[t,s] - b[t,s] * m.price[t])**2) / (2.0 * b[t,s]) for s in range(1,S)) # only commercial and industrial, residential fixed
    )
    m.cost_def = Constraint(
        m.T, rule = lambda m,t: m.costs[t] == sum(c[k] * m.q[t,k] for k in m.I)
                           + (m.imports[t] - data.am[t])**2 / (2.0 * data.bm[t])
    )

    # Capacity constraints
    m.cap_hydronuc = Constraint(m.T, expr={t: m.q[t,"hydronuc"] <= data.hydronuc[t] for t in m.T})
    if capUB:
        m.cap_gas123 = Constraint(m.T, Set(initialize=list(capUB)),
                                  expr={(t,k): m.q[t,k] <= capUB[k] for t in m.T for k in capUB})

    m.cap_newgas = Constraint(m.T, expr={t: m.q[t,"newgas"] <= m.K["newgas"] for t in m.T})
    m.cap_wind   = Constraint(m.T, expr={t: m.q[t,"wind"]   <= m.K["wind"]  * data.wind_cap[t]  for t in m.T})
    m.cap_solar  = Constraint(m.T, expr={t: m.q[t,"solar"]  <= m.K["solar"] * data.solar_cap[t] for t in m.T})

    res = SolverFactory(solver_name).solve(m, tee=False)
    term = str(res.solver.termination_condition)

    if term.lower() in ("optimal", "locallyoptimal", "locally_optimal", "locally optimal"):
        price   = np.array([value(m.price[t])   for t in m.T])
        demand  = np.array([value(m.demand[t,s])  for t in m.T for s in m.S]).reshape((T, S))
        imports = np.array([value(m.imports[t]) for t in m.T])
        q = {k: np.array([value(m.q[t,k]) for t in m.T]) for k in TECHS}
        K = {k: float(value(m.K[k]))           for k in INV_TECHS}

        w_arr = data["weights"].to_numpy()
        avg_price = float(np.sum(price * w_arr) / np.sum(w_arr))
        num = np.sum(w_arr[:, None] * price[:, None] * demand[:,0])
        den = np.sum(w_arr[:, None] * demand[:,0])
        avg_price_residential = num / den

        objective = float(np.sum([data.weights[t] * (value(m.gs[t]) - value(m.costs[t])) for t in m.T])
                        - F.get("newgas", 0.0) * K["newgas"]
                        - F.get("wind",   0.0) * K["wind"]
                        - F.get("solar",  0.0) * K["solar"])
        
        # emissions
        emissions = sum(w_arr[t] * sum(e[i] * q[i][t] for i in m.I) for t in m.T)

        # cost accounting
        gen_cost_t = np.array([value(m.costs[t])   for t in m.T])
        imp_cost_t = (imports - data["am"].to_numpy()) ** 2 / (2.0 * data["bm"].to_numpy())
        total_cost = float(np.sum(w_arr * (gen_cost_t + imp_cost_t))) + F.get("newgas", 0.0) * K["newgas"] +  F.get("wind",   0.0) * K["wind"] +  F.get("solar",  0.0) * K["solar"]

        return {"status": term, "avg_price": avg_price, "avg_price_residential": avg_price_residential, "price": price,
                "quantity": q, "imports": imports, "total_cost": total_cost,
                "demand": demand, "cost": total_cost, "objective": objective, "K": K, "emissions": emissions}

    return {"status": term}


In [179]:
res = clear_market_rooftop(dfclust,tech, nem=0)  # no solar panels
print(res["avg_price_residential"])
print(res["K"])

38.780473607394825
{'newgas': -9.404125952084079e-38, 'wind': 5.876766303734267e-39, 'solar': -9.975727450117208e-09}


Households should pay much more to cover for the fixed costs of the grid.

In [180]:
tariff = (np.sum(dfclust.weights * res["price"] * dfclust.q_residential) + grid_costs)/np.sum(dfclust.weights * dfclust.q_residential)
print(f"Retail tariff residential (with grid costs): {tariff:.2f} USD/MWh")

Retail tariff residential (with grid costs): 80.68 USD/MWh


## Finding the equilibrium

We need to solve for the level of investment consistent with zero profits as well as the equilibrium tariff for consumers.

Investment is solved by the function, the tariff is solved outside the model in line with previous simulations.

### Getting tariff as intermediate function
We create a function that solves the optimal tariff given how many consumers are contributing to recovery of grid costs, which depends on the net metering policy.

The function takes wholesale prices and solar investment as given.

The default recovers the previous result without netmetering.

Notice how net metering makes a very big difference.

In [181]:
def compute_tariff(data, prices, solar_gw=5.0, grid_cost=10000.0, nem=0, kw=3.0, hh=12.0):
      """
      Compute retail tariff for given prices and solar capacity
      This is not an equilibrium calculation!
      """
      data["selfc"] = np.minimum(data.solar_cap * kw, data.q_residential / hh)
      data["gout"]  = np.maximum(data.solar_cap * kw - data.q_residential / hh, 0.0)
      net_q = data.q_residential - data.selfc * solar_gw / kw
      num = (data.weights * prices * net_q).sum() + grid_cost
      den = (data.weights * (data.q_residential - data.solar_cap * solar_gw)).sum() if nem==1 else \
            (data.weights * net_q).sum() if nem in (2,3) else \
            (data.weights * data.q_residential).sum()

      return num / den

Without net-metering, everyone pays for their grid costs in proportion to their gross consumption.

In [182]:
net0 = compute_tariff(dfclust, res["price"], solar_gw=5.0, kw=3.0, grid_cost=grid_costs, nem=0)
print(f"Retail tariff residential no solar panels, no NEM: {net0:.2f} USD/MWh")

Retail tariff residential no solar panels, no NEM: 78.31 USD/MWh


Under net-metering 1.0, solar customers only pay net of their solar production. Thus, the tariff needs to be higher.

In [183]:
net1 = compute_tariff(dfclust, res["price"], solar_gw=5.0, kw=3.0, grid_cost=grid_costs, nem=1)
print(f"Retail tariff residential with solar panels, NEM: {net1:.2f} USD/MWh")

Retail tariff residential with solar panels, NEM: 85.80 USD/MWh


Under net-metering 2.0, we get a middle ground. Solar customers only pay for their consumption net of self-consumption. The residential tariff is in-between the two cases.

In [184]:
net2 = compute_tariff(dfclust, res["price"], solar_gw=5.0, kw=3.0, grid_cost=grid_costs, nem=2)
print(f"Retail tariff residential with solar panels, no NEM: {net2:.2f} USD/MWh")

Retail tariff residential with solar panels, no NEM: 82.53 USD/MWh


### Now we compute the market equilibrium by solving investment and guessing tariff

How does the net-metering scheme affect total investment? By allowing customers to net out their grid costs, net-metering 1.0 can make solar panels quite attractive.

To solve the equilibrium, we use the "solve_invest" function in the formula, and iterate to look for the equilibirum tariff, using the compute tariff function to know how much the tariff should be.

In [185]:
def clear_market_equilibrium(data, tech, grid_cost=10000.0, pretail=150.0, kw=3.0, hh=12.0, ng_price=4.0, nem=1):
    current_diff = 1.0
    guess = pretail
    while current_diff > 1e-2:
        res = clear_market_rooftop(data, tech, pretail=guess, kw=kw, hh=hh, ng_price=ng_price, nem=nem)
        if res["status"]  in ("optimal", "locallyoptimal", "locally_optimal", "locally optimal"):
            newguess = compute_tariff(data, res["price"], solar_gw=res["K"]["solar"], nem=nem, grid_cost=grid_cost)
        else:
            print(f"No solution, status {res['status']}, tariff guess {guess}")
            newguess = guess
        current_diff = (guess - newguess) ** 2
        guess = newguess

    # we solve at the equilibrium guess
    res = clear_market_rooftop(data, tech, pretail=guess, kw=kw, hh=hh, ng_price=ng_price, nem=nem)
    res["pretail"] = guess
    return res

Under net-metering 1.0, consumers can cancel their grid payments when producing with their solar power, regardless of whether they consume at the same time or not. Thus, we get more investment. It has great impacts on the retail price of consumers "left behind".

In this code, we only have one type of consumer, but in practice, some HHs put solar panels, and some don't, leading to very different retail prices.

In [190]:
res_eq1 = clear_market_equilibrium(dfclust, tech, nem = 1, grid_cost=grid_costs)
print("Number of adopting households (in million) ", int(res_eq1["K"]["solar"]/3.0))
print("Equilibrium retail tariff (with solar panels, NEM): ", res_eq1["pretail"])

Number of adopting households (in million)  12
Equilibrium retail tariff (with solar panels, NEM):  170.92873059341431


Under net-metering 2.0, consumers can only cancel their grid payments when producing with their solar power and consuming at the same time. Net-metering 1.0 acts as an implicit subsidy to solar panels.

In [187]:
res_eq2 = clear_market_equilibrium(dfclust, tech, nem = 2, grid_cost=grid_costs)
print("Number of adopting households (in million) ", res_eq2["K"]["solar"]/3.0)
print("Equilibrium retail tariff (with solar panels, NEM): ", res_eq2["pretail"])

Number of adopting households (in million)  2.742160830811278
Equilibrium retail tariff (with solar panels, NEM):  82.32260423654506


Residential solar can be more profitable than non-residential solar. For the equivalent non-residential solar outcome, we set nem to 0. In this case, we obtain zero investment without the additional incentive to avoid grid costs. 

Note: In practice, large scale solar is substantially cheaper, so both can be profitable.

In [194]:
res_eq0 = clear_market_equilibrium(dfclust, tech, nem = 0, grid_cost=grid_costs)
print("Number of adopting households (in million) ", int(res_eq0["K"]["solar"]/3.0))
print("Equilibrium retail tariff (with solar panels, NEM): ", res_eq0["pretail"])

Number of adopting households (in million)  0
Equilibrium retail tariff (with solar panels, NEM):  83.97439172304546


## Follow-up exercises

1. Think about how the code takes into account curtailment. Is it properly factored in?
   
2. Think about consumers being responsive. How would this affect the value of NEM 1.0 vs. NEM 2.0 or 3.0?
